In [1]:
%%capture
import sys

# Añade el directorio principal al path de búsqueda para importar módulos desde esa ubicación
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import tensorflow as tf
from packaging import version
from sklearn import datasets
from sklearn.model_selection import train_test_split

from likelihood.models.deep import AutoClassifier, AutoQL, Env
from likelihood.tools import OneHotEncoder, get_metrics

if version.parse(tf.__version__) > version.parse("2.15.0"):
    is_updated = True

2025-08-15 18:16:26.517420: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-15 18:16:26.521019: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-15 18:16:26.530203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755281786.544783  351270 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755281786.550606  351270 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755281786.563170  351270 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Cargar el dataset de cáncer de mama desde sklearn
df = datasets.load_breast_cancer()

# Convertir los datos a un DataFrame de pandas para facilitar la manipulación
df_cancer = pd.DataFrame(data=df.data, columns=df.feature_names)
df_cancer["target"] = df.target  # Añadir la columna de etiquetas 'target'
y = df_cancer["target"].to_list()

# OneHotEncoder convierte las etiquetas a formato one-hot encoding
encoder = OneHotEncoder()
y_enc = encoder.encode(y)  # Codificar las etiquetas de la clase (target)

X = df_cancer.drop(
    columns="target"
).to_numpy()  # Extraer las características (sin la columna 'target')
X = np.asarray(X).astype(np.float32)  # Convertir X a tipo float32 para la entrada del modelo
y = np.asarray(y).astype(np.float32)  # Convertir y a tipo float32

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = AutoClassifier(
    input_shape_parm=X.shape[1],  # El número de características de entrada (columnas de X)
    num_classes=y_enc.shape[1],  # El número de clases (salidas) del modelo
    units=17,  # Número de unidades en las capas ocultas
    activation="selu",  # Función de activación de las capas ocultas
    l2_reg=0.0,
)

if is_updated:
    model = model._main_model

print("Predictions before reinforcement learning:")
pred = model.predict(X)

# Obtener las etiquetas predichas para las nuevas predicciones
pred_label = np.argmax(pred, axis=1)

# Añadir las nuevas predicciones al DataFrame original
df = df_cancer.copy()
df["prediction"] = pred_label

# Calcular y mostrar las métricas nuevamente con el modelo cargado
get_metrics(df, "target", "prediction", verbose=True)

print("Reinforcement training:")
env = Env(model, 200)
q_agent = AutoQL(env, model)
q_agent.train(X_train, y_train)

Predictions before reinforcement learning:
 1/18 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step

2025-08-15 18:16:29.579829: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Accuracy: 37.26%
Precision: 0.00%
Recall: 0.00%
F1-Score: 0.00
Cohen's Kappa: 0.0000
Reinforcement training:
Episode: 0, Steps: 200, Epsilon: 1.000, Loss: 5.50e-06, Reward: 103, No Improve Count: 0
Episode: 5, Steps: 200, Epsilon: 0.875, Loss: 2.85e-06, Reward: 189, No Improve Count: 4
Episode: 10, Steps: 200, Epsilon: 0.750, Loss: 1.48e-06, Reward: 184, No Improve Count: 9
Early stopping at episode 11 due to no improvement in 10 episodes.


In [3]:
q_model = q_agent.model
# Hacer predicciones nuevamente con el modelo cargado
pred = q_model.predict(X)

# Obtener las etiquetas predichas para las nuevas predicciones
print("Predictions after reinforcement learning:")
pred_label = np.argmax(pred, axis=1)

# Añadir las nuevas predicciones al DataFrame original
df = df_cancer.copy()
df["prediction"] = pred_label

# Calcular y mostrar las métricas nuevamente con el modelo cargado
get_metrics(df, "target", "prediction", verbose=True)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Predictions after reinforcement learning:
Accuracy: 87.35%
Precision: 83.85%
Recall: 98.88%
F1-Score: 90.75
Cohen's Kappa: 0.7117
